# Packages 

In [ ]:
#package preparation
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
import os
from numpy import genfromtxt
from scipy import stats
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import keras
from keras.layers import Dense, Flatten, Reshape,Dropout
from keras.layers import Conv2D, MaxPooling2D, LSTM
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import timeit #package for recording the model running time
import time
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, Conv3D, MaxPooling3D, Reshape, BatchNormalization, MaxPooling2D
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
from sklearn.metrics import fbeta_score
from sklearn.model_selection import KFold,StratifiedKFold,ShuffleSplit,StratifiedShuffleSplit
# Packages & Setup
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import  preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,f1_score,accuracy_score
import timeit

# Functions 

In [ ]:
#function preparation
def win_seg(data,windowsize,overlap):#function for overlap segmentation
    length=int((data.shape[0]*data.shape[1]-windowsize)/(windowsize*overlap)+1)
    newdata=np.empty((length,windowsize, data.shape[2],1))
    data_dim=data.shape[2]
    layers=data.shape[3]
    data=data.reshape(-1,data_dim,layers)
    for i in range(0,length) :
        start=int(i*windowsize*overlap)
        end=int(start+windowsize)
        newdata[i]=data[start:end]
    return newdata
def lab_vote(data,windowsize):
    y_data=data.reshape(-1,windowsize,1,1)
    y_data=win_seg(y_data,windowsize,0.5)
    y_data=y_data.reshape(y_data.shape[0],y_data.shape[1],y_data.shape[2])
    y_data=stats.mode(y_data,axis=1)
    y_data=y_data.mode
    y_data=y_data.reshape(-1,1)
    y_data=np.float64(keras.utils.to_categorical(y_data))
    return y_data
def lab_vote_cat(data,windowsize): # non one-hot coding
    y_data=data.reshape(-1,windowsize,1,1)
    y_data=win_seg(y_data,windowsize,0.5)
    y_data=y_data.reshape(y_data.shape[0],y_data.shape[1],y_data.shape[2])
    y_data=stats.mode(y_data,axis=1)
    y_data=y_data.mode
    y_data=y_data.reshape(-1,1)
    return y_data
def write_csv(data):
    a = np.asarray(data)
    a.tofile('check.csv',sep=',',format='%10.5f')
def average(lst): 
    a = np.array(lst)
    return np.mean(a)
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []
    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()
    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# One-to-One Incremental Learning 

## Developing Incremental and Evaluating Incremental Performance

In [ ]:
#loading data for next subject, use S4 as an example
buffer = np.float64(preprocessing.scale(genfromtxt('S4_X.csv', delimiter=',')))
x_data=buffer.reshape(-1,40,30,1)
x_data=win_seg(x_data,40,0.5) # data segmentation with 0.5 overlap
#majority vote on training label
buffer = np.float64(genfromtxt('S4_Y.csv', delimiter=','))-1 #0 based index
y_data=lab_vote(buffer,40)
#fix the dimension mismatch
if y_data.shape[1]<7:
    zeros=np.zeros((y_data.shape[0],7-y_data.shape[1]))
    y_data=np.hstack((y_data,zeros))
y_data2=lab_vote_cat(buffer,40) # for stratification purposes
#Stratified random shuffle
sf = ShuffleSplit(n_splits=3, test_size=0.1, random_state=42)
sss=StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=42)

#Model Evaluation Metrics
acc_score=list()
f_score=list()
eopch_time_record=list()
oper_time_record=list()
i=0
for train_index, test_index in sss.split(x_data,y_data):
    #Reload the trained model for S3
    newmodel=load_model("S3%s.h5" % i, custom_objects={'f1': f1})
    X_train, X_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    #split the train data into training (training the model) and validation (tuning hypeparameters) by 8:2
    X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20)
    #Setup Model Parameters
    data_dim = X_train.shape[2] #y of figure
    timesteps = X_train.shape[1] #x of figure
    num_classes = y_train.shape[1]
    batchsize=300
    epcoh=300
    checkpointer = ModelCheckpoint(filepath="S4%s.h5" % i, monitor='val_f1',verbose=1, mode='max', save_best_only=True)
    time_callback = TimeHistory() #record the model training time for each epoch
    callbacks_list = [checkpointer,time_callback]
    newmodel.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy',f1])
    train_history=newmodel.fit(X_training, y_training,
              batch_size=batchsize, epochs=epcoh,callbacks=callbacks_list,
              validation_data=(X_validation, y_validation))
    eopch_time=time_callback.times
    eopch_time_record.append(eopch_time) #record the traing time of each epoch
    CNN_LSTM_model=load_model("S4%s.h5" % i, custom_objects={'f1': f1})
    #model operation and timing
    start=timeit.default_timer()
    y_pred=CNN_LSTM_model.predict(X_test)
    stop=timeit.default_timer()
    oper_time=stop-start
    oper_time_record.append(oper_time)
    #check the model test result
    y_pred=CNN_LSTM_model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    Y_test=np.argmax(y_test, axis=1)
    acc_score.append(accuracy_score(Y_test, y_pred)) # Evaluation of accuracy
    f_score.append(f1_score(Y_test, y_pred,average='macro')) # Evaluation of F1 score
    print("This is the", i+1,  "out of ",5, "Shuffle")
    i+=1
# Record incremetnal performance
performance=pd.DataFrame(columns=['Acc_score','Macro_Fscore','Average_Epoch','Average_Run'])
performance['Acc_score']=acc_score
performance['Macro_Fscore']=f_score
performance['Average_Epoch']=average(eopch_time_record)
performance['Average_Run']=average(oper_time_record)
performance.to_csv("Incremental Performance on S4.csv")

## Evaluating Forgetting Efffect

In [ ]:
#loading data of learned subject
buffer = np.float64(preprocessing.scale(genfromtxt('S3_X.csv', delimiter=',')))
x_data=buffer.reshape(-1,40,30,1)
x_data=win_seg(x_data,40,0.5) # data segmentation with 0.5 overlap
#majority vote on training label
buffer = np.float64(genfromtxt('S3_Y.csv', delimiter=','))-1 #0 based index
y_data=lab_vote(buffer,40)
#fix the dimension mismatch
if y_data.shape[1]<7:
    zeros=np.zeros((y_data.shape[0],7-y_data.shape[1]))
    y_data=np.hstack((y_data,zeros))
y_data2=lab_vote_cat(buffer,40) # for stratification purposes
#stratified random shaffle
sf = ShuffleSplit(n_splits=3, test_size=0.1, random_state=42)
sss=StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=42)

#Model Evaluation Metrics
acc_score=list()
f_score=list()
eopch_time_record=list()
oper_time_record=list()
i=0
for train_index, test_index in sss.split(x_data,y_data):
    X_train, X_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    #split the train data into training (training the model) and validation (tuning hypeparameters) by 8:2
    X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20)
    #Setup Model Parameters
    data_dim = X_train.shape[2] #y of figure
    timesteps = X_train.shape[1] #x of figure
    num_classes = y_train.shape[1]
    batchsize=300
    epcoh=300
    #Load Model
    CNN_LSTM_model=load_model("S4.h5" % i, custom_objects={'f1': f1})
    #model operation and timing
    start=timeit.default_timer()
    y_pred=CNN_LSTM_model.predict(X_test)
    stop=timeit.default_timer()
    oper_time=stop-start
    oper_time_record.append(oper_time)
    #check the model test result
    y_pred=CNN_LSTM_model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    Y_test=np.argmax(y_test, axis=1)
    acc_score.append(accuracy_score(Y_test, y_pred)) # Evaluation of accuracy
    f_score.append(f1_score(Y_test, y_pred,average='macro')) # Evaluation of F1 score
    print("This is the", i+1,  "out of ",5, "Shuffle")
    i+=1
    
# Record Performance
performance=pd.DataFrame(columns=['Acc_score','Macro_Fscore','Average_Epoch','Average_Run'])
performance['Acc_score']=acc_score
performance['Macro_Fscore']=f_score
performance['Average_Epoch']=average(eopch_time_record)
performance['Average_Run']=average(oper_time_record)
performance.to_csv("[Evaluation of Forget on S3].csv")

# Many-to-One Incremental Learning  

## Developing Incremental and Evaluating Incremental Performance

In [ ]:
#loading data for next subject, use S4 as an example
buffer = np.float64(preprocessing.scale(genfromtxt('S4_X.csv', delimiter=',')))
x_data=buffer.reshape(-1,40,30,1)
x_data=win_seg(x_data,40,0.5) # data segmentation with 0.5 overlap
#majority vote on training label
buffer = np.float64(genfromtxt('S4_Y.csv', delimiter=','))-1 #0 based index
y_data=lab_vote(buffer,40)
#fix the dimension mismatch
if y_data.shape[1]<7:
    zeros=np.zeros((y_data.shape[0],7-y_data.shape[1]))
    y_data=np.hstack((y_data,zeros))
y_data2=lab_vote_cat(buffer,40) # for stratification purposes
#Stratified random shuffle
sf = ShuffleSplit(n_splits=3, test_size=0.1, random_state=42)
sss=StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=42)

#Model Evaluation Metrics
acc_score=list()
f_score=list()
eopch_time_record=list()
oper_time_record=list()
i=0
for train_index, test_index in sss.split(x_data,y_data):
    #Reload the trained model for rest subjects
    newmodel=load_model("rest%s.h5" % i, custom_objects={'f1': f1})
    X_train, X_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    #split the train data into training (training the model) and validation (tuning hypeparameters) by 8:2
    X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20)
    #Setup Model Parameters
    data_dim = X_train.shape[2] #y of figure
    timesteps = X_train.shape[1] #x of figure
    num_classes = y_train.shape[1]
    batchsize=300
    epcoh=300
    checkpointer = ModelCheckpoint(filepath="S4%s.h5" % i, monitor='val_f1',verbose=1, mode='max', save_best_only=True)
    time_callback = TimeHistory() #record the model training time for each epoch
    callbacks_list = [checkpointer,time_callback]
    newmodel.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy',f1])
    train_history=newmodel.fit(X_training, y_training,
              batch_size=batchsize, epochs=epcoh,callbacks=callbacks_list,
              validation_data=(X_validation, y_validation))
    eopch_time=time_callback.times
    eopch_time_record.append(eopch_time) #record the traing time of each epoch
    CNN_LSTM_model=load_model("S4%s.h5" % i, custom_objects={'f1': f1})
    #model operation and timing
    start=timeit.default_timer()
    y_pred=CNN_LSTM_model.predict(X_test)
    stop=timeit.default_timer()
    oper_time=stop-start
    oper_time_record.append(oper_time)
    #check the model test result
    y_pred=CNN_LSTM_model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    Y_test=np.argmax(y_test, axis=1)
    acc_score.append(accuracy_score(Y_test, y_pred)) # Evaluation of accuracy
    f_score.append(f1_score(Y_test, y_pred,average='macro')) # Evaluation of F1 score
    print("This is the", i+1,  "out of ",5, "Shuffle")
    i+=1
# Record incremetnal performance
performance=pd.DataFrame(columns=['Acc_score','Macro_Fscore','Average_Epoch','Average_Run'])
performance['Acc_score']=acc_score
performance['Macro_Fscore']=f_score
performance['Average_Epoch']=average(eopch_time_record)
performance['Average_Run']=average(oper_time_record)
performance.to_csv("Incremental Performance on S4.csv")

## Evaluating Forgetting Efffect

In [ ]:
#loading data of learned (rest) subjects
buffer = np.float64(preprocessing.scale(genfromtxt('rest_X.csv', delimiter=',')))
x_data=buffer.reshape(-1,40,30,1)
x_data=win_seg(x_data,40,0.5) # data segmentation with 0.5 overlap
#majority vote on training label
buffer = np.float64(genfromtxt('rest_Y.csv', delimiter=','))-1 #0 based index
y_data=lab_vote(buffer,40)
#fix the dimension mismatch
if y_data.shape[1]<7:
    zeros=np.zeros((y_data.shape[0],7-y_data.shape[1]))
    y_data=np.hstack((y_data,zeros))
y_data2=lab_vote_cat(buffer,40) # for stratification purposes
#stratified random shaffle
sf = ShuffleSplit(n_splits=3, test_size=0.1, random_state=42)
sss=StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=42)

#Model Evaluation Metrics
acc_score=list()
f_score=list()
eopch_time_record=list()
oper_time_record=list()
i=0
for train_index, test_index in sss.split(x_data,y_data):
    X_train, X_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    #split the train data into training (training the model) and validation (tuning hypeparameters) by 8:2
    X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20)
    #Setup Model Parameters
    data_dim = X_train.shape[2] #y of figure
    timesteps = X_train.shape[1] #x of figure
    num_classes = y_train.shape[1]
    batchsize=300
    epcoh=300
    #Load the Trained Model
    CNN_LSTM_model=load_model("S4.h5" % i, custom_objects={'f1': f1})
    #model operation and timing
    start=timeit.default_timer()
    y_pred=CNN_LSTM_model.predict(X_test)
    stop=timeit.default_timer()
    oper_time=stop-start
    oper_time_record.append(oper_time)
    #check the model test result
    y_pred=CNN_LSTM_model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    Y_test=np.argmax(y_test, axis=1)
    acc_score.append(accuracy_score(Y_test, y_pred)) # Evaluation of accuracy
    f_score.append(f1_score(Y_test, y_pred,average='macro')) # Evaluation of F1 score
    print("This is the", i+1,  "out of ",5, "Shuffle")
    i+=1
    
# Record Performance
performance=pd.DataFrame(columns=['Acc_score','Macro_Fscore','Average_Epoch','Average_Run'])
performance['Acc_score']=acc_score
performance['Macro_Fscore']=f_score
performance['Average_Epoch']=average(eopch_time_record)
performance['Average_Run']=average(oper_time_record)
performance.to_csv("[Evaluation of Forget on Rest].csv")